In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from tqdm.auto import tqdm
from util.labelling_helper import GroundTruthLabellingHelper
tqdm.pandas()


In [25]:
df = pd.read_csv('data/acl/sections/sections_clean.csv')
df.parent_header.fillna('', inplace=True)

In [26]:
df["content_length"] = df.content.apply(lambda x: len(x.split(" ")))
metrics = df.content_length.describe()
metrics

count    35482.000000
mean        52.077194
std         92.840608
min          1.000000
25%         14.000000
50%         31.000000
75%         65.000000
max      11207.000000
Name: content_length, dtype: float64

In [13]:
upper_median_df = df.loc[df.content_length.apply(lambda x: (x < metrics["mean"] + metrics["std"]))]
upper_median_df.shape


(33003, 8)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('all-mpnet-base-v2', device=device)
labelling_helper = GroundTruthLabellingHelper(model)


In [ ]:
labelled_df = upper_median_df.copy()
labelled_df["text"] = labelled_df[["parent_header", "header", "content"]].apply(
    lambda x: '\n'.join(filter(None, x)), axis=1)
labelled_df[["label", "score"]] = labelled_df["text"].progress_apply(
    lambda x: pd.Series(labelling_helper.get_most_similar_group(x)))


In [ ]:
labelled_df.score.describe()


In [ ]:
labelled_df = df.copy()
labelled_df["text"] = labelled_df[["parent_header", "header", "content"]].apply(
    lambda x: '\n'.join(filter(None, x)), axis=1)
labelled_df[["label", "score"]] = labelled_df["text"].progress_apply(
    lambda x: pd.Series(labelling_helper.get_most_similar_group(x)))


In [ ]:
labelled_df.score.describe()


In [ ]:
upper_mean_df = df.loc[df.content.apply(
    lambda x: (len(x) > content_length["mean"]))]


In [ ]:
labelled_df = upper_mean_df.copy()
labelled_df["text"] = labelled_df[["parent_header", "header", "content"]].apply(
    lambda x: '\n'.join(filter(None, x)), axis=1)
labelled_df[["label", "score"]] = labelled_df["text"].progress_apply(
    lambda x: pd.Series(labelling_helper.get_most_similar_group(x)))


In [ ]:
labelled_df.score.describe()


In [ ]:
labelled_df.groupby("label").agg({"score": ["mean", "count"]})
